In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import xesmf as xe
from tqdm.autonotebook import tqdm  # Fancy progress bars for our loops!
import fsspec
import cftime
from dask import optimize
from scipy.integrate import odeint
from scipy.optimize import minimize
from scipy.interpolate import interp1d
from sklearn.linear_model import LinearRegression
import pywt

/var/folders/7y/lr_w90f56s35gmw3l6ghkz_w0000gn/T/ipykernel_59435/3185945064.py:6: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm  # Fancy progress bars for our loops!


In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.rcParams['figure.figsize'] = 12, 6
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)

df = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')

In [3]:
df_area = df.query("variable_id == 'areacella' & source_id == 'CESM2' & experiment_id =='historical'")
ds_area = xr.open_zarr(fsspec.get_mapper(df_area.zstore.values[-1]), consolidated=True, chunks=None)
total_area = float(ds_area.areacella.sum(dim=['lon', 'lat']))

In [4]:
ds_area

<xarray.Dataset>
Dimensions:    (lat: 192, lon: 288, nbnd: 2)
Coordinates:
  * lat        (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
    lat_bnds   (lat, nbnd) float64 ...
  * lon        (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
    lon_bnds   (lon, nbnd) float64 ...
Dimensions without coordinates: nbnd
Data variables:
    areacella  (lat, lon) float32 2.995e+07 2.995e+07 ... 2.995e+07 2.995e+07
Attributes: (12/47)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   674885.0
    branch_time_in_parent:  219000.0
    case_id:                972
    ...                     ...
    variable_id:            areacella
    variant_info:           CMIP6 20th century experiments (1850-2014) with C...
    variant_label:          r11i1p1f1
    status:                 2019-10-25;created;by nhn2@columbia.edu
    netcdf_tracking_ids:    hdl:21.14100/96455df2-979e-4cd4-8521-ddf307c6bc4a
    version_id:             v20190514

In [5]:
# df_ta_ncar = df.query("activity_id=='CMIP' & table_id == 'Amon' &  source_id == 'CESM2' & variable_id == 'tas'  & institution_id == 'NCAR'")
df_EW_ncar_his = df.query("activity_id=='CMIP' & table_id == 'Amon' &  source_id == 'CESM2'  & variable_id == 'ua' & institution_id == 'NCAR' & experiment_id == 'historical'")
zstore_his = df_EW_ncar_his.zstore.values[-1]
print(zstore_his)
mapper_his = fsspec.get_mapper(zstore_his)
ds_EW_his = xr.open_zarr(mapper_his, consolidated=True)
display(ds_EW_his)

# df_NW_ncar_his = df.query("activity_id=='CMIP' & table_id == 'Amon' &  source_id == 'CESM2'  & variable_id == 'va' & institution_id == 'NCAR' & experiment_id == 'historical'")
# zstore_his = df_NW_ncar_his.zstore.values[-1]
# print(zstore_his)
# mapper_his = fsspec.get_mapper(zstore_his)
# ds_NW_his = xr.open_zarr(mapper_his, consolidated=True, chunks=None)
# display(ds_NW_his)


time = np.arange(1850, 2015, 1/12)
# date_threshold = cftime.DatetimeNoLeap(1550, 1, 1)
# ds_tas_his_plt = ds_tas_his.where(ds_tas_his.time > date_threshold, drop=True)
# # weighted product
# tas_timeseries_his = (ds_tas_his_plt.tas * ds_area.areacella).sum(dim=['lon', 'lat']) / total_area
# # convert to np array
# array_tas_ts = np.array(tas_timeseries_his)
# # plt.plot(array_tas_ts)
# # plt.show()
# year_tas_ts = np.mean(array_tas_ts.reshape(1980//12, 12), axis=1)
# # plt.plot(year_tas_ts)
# # plt.show()

gs://cmip6/CMIP6/CMIP/NCAR/CESM2/historical/r11i1p1f1/Amon/ua/gn/v20190514/


<xarray.Dataset>
Dimensions:    (lat: 192, nbnd: 2, lon: 288, plev: 19, time: 1980)
Coordinates:
  * lat        (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
    lat_bnds   (lat, nbnd) float64 dask.array<chunksize=(192, 2), meta=np.ndarray>
  * lon        (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
    lon_bnds   (lon, nbnd) float64 dask.array<chunksize=(288, 2), meta=np.ndarray>
  * plev       (plev) float64 1e+05 9.25e+04 8.5e+04 7e+04 ... 1e+03 500.0 100.0
  * time       (time) object 1850-01-15 12:00:00 ... 2014-12-15 12:00:00
    time_bnds  (time, nbnd) object dask.array<chunksize=(1980, 2), meta=np.ndarray>
Dimensions without coordinates: nbnd
Data variables:
    ua         (time, plev, lat, lon) float32 dask.array<chunksize=(60, 19, 192, 288), meta=np.ndarray>
Attributes: (12/49)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   674885.0
    branch_time_in_parent:  219000.0
    case_id:                972
    ...                     ...
    variable_id:            ua
    variant_info:           CMIP6 20th century experiments (1850-2014) with C...
    variant_label:          r11i1p1f1
    status:                 2019-10-25;created;by nhn2@columbia.edu
    netcdf_tracking_ids:    hdl:21.14100/05f2f382-e465-4c2b-9a09-e5e9d4e45c19...
    version_id:             v20190514

## Selected latitude range from 30 to 60 North. All longitudes

In [6]:
wind_u = ds_EW_his.sel(plev=20000)
# wind_v = ds_NW_his.sel(plev=20000)

lat_range = ds_EW_his.lat.sel(lat=slice(30, 60))
lon_range = ds_EW_his.lon.sel(lon=slice(0, 360))
# Here I am taking the mean
wind_u_timeseries = (wind_u.ua * ds_area.areacella).sel(lat=lat_range, lon=lon_range).sum(dim=["lat", "lon"]) / total_area
# wind_v_timeseries = (wind_v.va * ds_area.areacella).sel(lat=lat_range, lon=lon_range).sum(dim=["lat", "lon"])

In [7]:
wind_timeseries = wind_u_timeseries.chunk({'time': -1})
wind_timeseries

<xarray.DataArray (time: 1980)>
dask.array<rechunk-merge, shape=(1980,), dtype=float32, chunksize=(1980,), chunktype=numpy.ndarray>
Coordinates:
    plev     float64 2e+04
  * time     (time) object 1850-01-15 12:00:00 ... 2014-12-15 12:00:00

In [8]:
wind_timeseries.data.compute()

array([4.8974795, 4.3345866, 4.2461724, ..., 4.033554 , 4.8906226,
       5.0732493], dtype=float32)

In [35]:
wind_clean = wind_timeseries.reset_coords(drop=True)
wind_clean

<xarray.DataArray (time: 1980)>
dask.array<rechunk-merge, shape=(1980,), dtype=float32, chunksize=(1980,), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) object 1850-01-15 12:00:00 ... 2014-12-15 12:00:00

In [37]:
import dask
dask.config.set(scheduler='synchronous')
print(dask.config.get('scheduler'))


synchronous


In [23]:
wind_np_using_clean = wind_clean.compute()

In [36]:
wdnp = wind_clean.data.compute()
wdnp

array([4.8974795, 4.3345866, 4.2461724, ..., 4.033554 , 4.8906226,
       5.0732493], dtype=float32)

In [9]:
date_threshold = cftime.DatetimeNoLeap(1850, 2, 1)
test1 = ds_EW_his.where(ds_EW_his.time < date_threshold, drop=True)
test1 = test1.sel(plev = 20000)

test2 = (test1.ua * ds_area.areacella).sel(lat=lat_range, lon=lon_range).sum(dim=["lat", "lon"]) / total_area
test2 = test2.reset_coords(drop=True)
test2

<xarray.DataArray (time: 1)>
dask.array<truediv, shape=(1,), dtype=float32, chunksize=(1,), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) object 1850-01-15 12:00:00

In [10]:
test2.values

array([4.8974795], dtype=float32)

In [80]:
print(type(test1.ua.data))
print(type(ds_area.areacella.data))
print(float(total_area))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
510099676200960.0


In [85]:
test2.values

array([4.8974795], dtype=float32)

In [87]:
ua_weighted = (test1.ua * ds_area.areacella).compute()  # Compute multiplication first
ua_weighted_subset = ua_weighted.sel(lat=lat_range, lon=lon_range).compute()  # Compute selection
test2 = ua_weighted_subset.sum(dim=["lat", "lon"]) / total_area

### Don't re-run this cell if lat and lon range not changed. The runtime is 6.5 minutes

In [ ]:
# wind_timeseries = wind_timeseries.chunk({'time': 1980})
# wind_timeseries = wind_timeseries.persist()
# wind_timeseries

wind_u_timeseries = wind_u_timeseries.squeeze()  # Removes the 'plev' axis if it's size 1
wind_u_timeseries = wind_u_timeseries.compute()  # Forces computation to memory
wind_u_np = wind_u_timeseries.values  # Fastest way to get NumPy array

# wind_v_timeseries = wind_v_timeseries.squeeze()  # Removes the 'plev' axis if it's size 1
# wind_v_timeseries = wind_v_timeseries.compute()  # Forces computation to memory
# wind_v_np = wind_v_timeseries.values  # Fastest way to get NumPy array